# Permutations of a Simple Circuit

This notebook walks through how to utilize the core semantics of SysML v2 to generate alternative circuits as inputs to an OpenMDAO solution of these circuits. 

## Background

The M1 user model in SysML v2 is meant to be a set of constraints and rules under which legal instances can be created. Those instances should be taken as alternative produced systems and they can be analyzed in that way.

## Libraries Load-Up

Load up PyMBE and its various libraries.

In [ ]:
from pathlib import Path
import networkx as nx
import matplotlib as plt

import pymbe.api as pm

from pymbe.client import SysML2Client
from pymbe.graph.lpg import SysML2LabeledPropertyGraph
from pymbe.interpretation.interpretation import repack_instance_dictionaries
from pymbe.interpretation.interp_playbooks import (
    build_expression_sequence_templates,
    build_sequence_templates,
    random_generator_playbook,
    random_generator_phase_1_multiplicities,
)
from pymbe.interpretation.results import *
from pymbe.label import get_label_for_id
from pymbe.query.metamodel_navigator import feature_multiplicity
from pymbe.query.query import (
    roll_up_multiplicity,
    roll_up_upper_multiplicity,
    roll_up_multiplicity_for_type,
    get_types_for_feature,
    get_features_typed_by_type,
)
from pymbe.local.stablization import build_stable_id_lookups

## Load Up Model

Read the model from the local JSON file.

In [ ]:
circuit_client = SysML2Client()

circuit_file = Path(pm.__file__).parent / "../../tests/fixtures/Circuit Builder.json"

circuit_client._load_from_file(circuit_file)

circuit_lpg = SysML2LabeledPropertyGraph()
circuit_lpg.model = circuit_client.model

circuit_model = circuit_lpg.model

[id_to_circuit_name_lookup, circuit_name_to_id_lookup] = build_stable_id_lookups(circuit_lpg)

circuit_lpg.model.MAX_MULTIPLICITY = 10

## Explore Contents of Model with M1 in Memory

Use the M1 memory objects to see what is in the current model, starting with the main packages.

In [ ]:
circuit_model.packages

In [ ]:
circuit_model.ownedElement["Circuit Builder"].ownedElement

In [ ]:
circuit_def = circuit_model.ownedElement["Circuit Builder"].ownedElement["Circuit"]

### Circuit and its Features

Here is the circuit and its features, both parts and used connections.

In [ ]:
circuit_def.relationships

In [ ]:
circuit_def.ownedMember

In [ ]:
circuit_def.ownedMember[7].name

In [ ]:
circuit_def.ownedMember[7].multiplicity.lowerBound

In [ ]:
circuit_def.ownedMember[7].multiplicity.upperBound

## Generate M0 instances from the M1 model

Use the M1 model to start creating a series of instances to represent the circuits that should be analyzed.

In [ ]:
m0_interpretations = [
    random_generator_playbook(
        lpg=circuit_lpg,
        name_hints={},
        filtered_feat_packages=[circuit_lpg.model.ownedElement["Circuit Builder"]],
        phase_limit=10
    ) for run in range(0,100)
]

In [ ]:
pprint_interpretation(m0_interpretations[0], circuit_model, False)

## Filter M0 Instances for Reasonable Circuits

Until we get more sophisticated and can interpret constraints, the initial approach is to filter out solutions with unanalyzable layouts or trim the layouts to something more tractable.

### Connector End Checks

Look at the ends of the three main kinds of connectors.

In [ ]:
n2n = circuit_def.ownedMember["Node to Node"]

In [ ]:
p2p = circuit_def.ownedMember["Part to Part"]

In [ ]:
p2n = circuit_def.ownedMember["Part to Node"]

In [ ]:
circuit_def.ownedMember["Part to Part"].endFeature[0]._id

In [ ]:
m0_interpretations[10][p2n.endFeature[0]._id]

In [ ]:
m0_interpretations[10][p2n.endFeature[1]._id]

In [ ]:
m0_interpretations[10][p2p.endFeature[0]._id]

In [ ]:
m0_interpretations[10][p2p.endFeature[1]._id]

In [ ]:
m0_interpretations[10][n2n.endFeature[0]._id]

In [ ]:
m0_interpretations[10][n2n.endFeature[1]._id]

# OpenMDAO
> Based on OpenMDAO's [nonlinear circuit analysis example](https://openmdao.org/newdocs/versions/latest/examples/circuit_analysis_examples.html).

In [ ]:
import openmdao.api as om
from openmdao.test_suite.test_examples.test_circuit_analysis import Diode, Node, Resistor

In [ ]:
class Circuit(om.Group):

    def setup(self):
        self.add_subsystem('n1', Node(n_in=1, n_out=2), promotes_inputs=[('I_in:0', 'I_in')])
        self.add_subsystem('n2', Node())  # leaving defaults

        self.add_subsystem('R1', Resistor(R=100.), promotes_inputs=[('V_out', 'Vg')])
        self.add_subsystem('R2', Resistor(R=10000.))
        self.add_subsystem('D1', Diode(), promotes_inputs=[('V_out', 'Vg')])

        self.connect('n1.V', ['R1.V_in', 'R2.V_in'])
        self.connect('R1.I', 'n1.I_out:0')
        self.connect('R2.I', 'n1.I_out:1')

        self.connect('n2.V', ['R2.V_out', 'D1.V_in'])
        self.connect('R2.I', 'n2.I_in:0')
        self.connect('D1.I', 'n2.I_out:0')

        self.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
        self.nonlinear_solver.options['iprint'] = 2
        self.nonlinear_solver.options['maxiter'] = 20
        self.linear_solver = om.DirectSolver()

In [ ]:
p = om.Problem()
model = p.model

model.add_subsystem('ground', om.IndepVarComp('V', 0., units='V'))
model.add_subsystem('source', om.IndepVarComp('I', 0.1, units='A'))
model.add_subsystem('circuit', Circuit())

model.connect('source.I', 'circuit.I_in')
model.connect('ground.V', 'circuit.Vg')

p.setup()

# set some initial guesses
p['circuit.n1.V'] = 12.
p['circuit.n2.V'] = 1.

p.run_model()

print(p['circuit.n1.V'])
print(p['circuit.n2.V'])
print(p['circuit.R1.I'])
print(p['circuit.R2.I'])
print(p['circuit.D1.I'])

# sanity check: should sum to .1 Amps
assert p['circuit.R1.I'] + p['circuit.D1.I'] == 0.1

In [ ]:
elements = circuit_model.elements
sequences = m0_interpretations[20]

id_ = "99db05a8-32cd-4676-a936-0bc13bb6ec29"
id_ = "2ac62b51-dabf-4516-b56e-84e77ef3a67a"
id_ = "06fb7cf9-09e0-43c0-84a8-a5ac334b0b51"
id_ = "8dbe719d-b4e1-47b6-a40d-4c5fcfd30eba"
id_ = "35cb79bd-c6fa-400b-a37a-3ab28793d996"

elements[id_].owner.ownedElement, sequences[id_], sequences["11e7ca81-539a-414e-b59b-96c5c34d7aa4"]

In [ ]:
m0_interpretations[0]